<a href="https://colab.research.google.com/github/julianovale/PO240_Meta_heuristica/blob/main/PO240_Semana07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
# importações
import matplotlib.pyplot as plt
import random 
import numpy as np
import time 
import copy
import imageio
import os

def cria_gif(arquivos, nome_gif):
  imagens = []
  for arquivo_png in arquivos:
      imagens.append(imageio.imread(arquivo_png))
  duration = 1
  imageio.mimwrite(nome_gif, imagens, duration=duration, loop=3)
  for arquivo_png in arquivos:
      os.remove(arquivo_png)
        
def grafico(populacao, TP, g, arquivos):
  fig, ax = plt.subplots(figsize=(10,4))       # cria objeto figura e os eixos da figura
  
  # funções de uma variável y = f(x)
  x = np.arange(0, 31.0, 0.1) 
  y = funcao(x)
  ax.plot(x, y)  # estabelece a ordem dos eixos e faz o gráfico
  
  for i in range(TP):
      xi = populacao[i][1]
      yi = populacao[i][2]
      ax.plot([xi],[yi], "ro")       
  ax.grid(True, linestyle='-')  # formato da grade de fundo (opcional)
  nome = "GA"+str(g)+".png"
  arquivos.append(nome)
  fig.savefig(nome) #, dpi=fig.dpi)
  plt.close(fig)
  
def funcao(x):
  return np.sin(np.pi*x) - (1/20)*x**2 + x

def criaIndividuo(N): # cria uma cadeia binária aleatória
  individuo = [random.randint(0,1) for b in range(N)]
  return individuo

def criaPopulacao(TP, N):
  # [[lista binária], inteiro decodificado, função objetivo]
  # 1 indivíduo: [[1,1,1,1,1], 31, -17.049999999999997]
  populacao = [[criaIndividuo(N), 0, 0] for i in range(TP)]
  return populacao

def decodificaIndividuo(individuo, N):
  inteiro = 0

  for b in range(N):
    bit = individuo[b]
    potencia = N - 1 - b
    inteiro = inteiro + bit*(2**potencia)

  return inteiro

def avaliaPopulacao(populacao, N, TP):
  for i in range(TP):
    inteiro = decodificaIndividuo(populacao[i][0], N) # só a cadeia
    fitness = funcao(inteiro)
    populacao[i][1] = inteiro
    populacao[i][2] = fitness
        
def main():
  # constantes
  N = 5          # quantidade de bits de um indivíduo
  TP = 2        # tamanho da população do GA
  G = 2          # número de gerações do GA
  random.seed()  # inicia a semente dos número pseudo randômicos
  arquivos = []
  nome = "filme.gif"
  
  inicio = time.time()  # começa a contar o tempo
  
  # Inicializa População 
  populacao = criaPopulacao(TP, N)
  

  # Avalia População
  avaliaPopulacao(populacao, N, TP)
  grafico(populacao, TP, 0, arquivos)
  print(populacao)
  
#     for g in range(1, G):  # gerações
      # seleciona pares de pais aleatoriamente
      # crossover
      # mutação
      # com filhos: Avalia
      # populacao próxima geração, seleção: pais e filhos
      
  # fim do GA
  fim = time.time()
  print("Tempo de execução (s): ", fim - inicio)
#     cria_gif(arquivos, nome)

In [ ]:
main()

[[[1, 1, 1, 0, 1], 29, -13.050000000000004], [[1, 0, 1, 0, 1], 21, -1.0500000000000007]]
Tempo de execução (s):  0.0895528793334961
